# Inference Demo: Predicting Plant Health

This notebook demonstrates how to use a trained model to perform inference on sample images from the PlantVillage dataset. We will load the model, make predictions on a few images, and display the results.

## Step 1: Setup

In [ ]:
import sys

sys.path.append('..')

from src import config
from src.infer import load_model, predict_single_image

## Step 2: Load Model and Make Predictions

In [ ]:
model = load_model()

# Use the PlantVillage validation directory
val_dir = config.DATA_PROCESSED_DIR / "PlantVillage" / "val"
assert val_dir.exists(), f"Validation directory not found: {val_dir}"

# Build class names from the validation directory
class_names = sorted([d.name for d in val_dir.iterdir() if d.is_dir()])
print(f"Found {len(class_names)} classes: {class_names[:5]}...")

# Pick a few images to test
sample_images = list(val_dir.rglob("*.jpg"))[:5]
print(f"\nTesting on {len(sample_images)} sample images:")

for img_path in sample_images:
    result = predict_single_image(model, img_path, class_names)
    print(f"\nImage: {img_path.name}")
    print(f"Predicted: {result.get('predicted_class', 'Unknown')} (confidence: {result['confidence']:.3f})")
    print(f"Health: {result.get('health', 'n/a')}")
    print(f"True class: {img_path.parent.name}")
    print(f"Correct: {result.get('predicted_class') == img_path.parent.name}")